**PROYECTO:** Detección de Spam con Regresión Logística
* Mediante el flujo típico en competencias/text classification:
1. Cargar y explorar datos
2. Preprocesamiento de texto (limpieza, tokenización)
3. Representación (TF-IDF)
4. División train/test y balanceo de clases
5. Entrenamiento con regresión logística
6. Evaluación de métricas (accuracy, precisión, recall, F1, matriz de confusión, curva ROC)
7. Predicción sobre test y creación de submission.csv

Se usará el **dataset clásico SMS Spam Collection** (spam.csv, disponible en Kaggle: SMS Spam Collection Dataset (https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)
).

**Objetivo:** 
Construir un modelo de clasificación binaria para identificar si un mensaje SMS es spam (1) o ham (0).

**Dataset:** 
SMS Spam Collection (columna `label` con valores `spam`/`ham`, columna `message` con el texto).

# 1) Instalación de librerías (si falta algo)

In [ ]:

# !pip install pandas numpy matplotlib seaborn scikit-learn nltk wordcloud --quiet

# 2) Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

import nltk
from nltk.corpus import stopwords
import string
import re
from wordcloud import WordCloud
import warnings
warnings.filterwarnings("ignore")
nltk.download("stopwords")

# 3) Cargar datos

In [ ]:
df = pd.read_csv("spam.csv", encoding="latin-1")[["v1", "v2"]]
df.columns = ["label","message"]
print("Shape:", df.shape)
df.head()

# 4) Exploración de datos
- Revisar balance de clases.

In [ ]:
df["label"].value_counts()
sns.countplot(data=df, x="label")
plt.title("Distribución de clases (spam vs ham)")
plt.show()

df["label"] = df["label"].map({"ham":0,"spam":1})

# 5) Limpieza de texto
- Pasar a minúsculas
- Quitar puntuación, números y stopwords

In [ ]:
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = " ".join([w for w in text.split() if w not in stop_words])
    return text

df["clean_msg"] = df["message"].apply(clean_text)
df.head()

# 6) Visualización rápida con WordCloud

In [ ]:
spam_words = " ".join(df[df["label"]==1]["clean_msg"])
ham_words = " ".join(df[df["label"]==0]["clean_msg"])

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
WordCloud(width=500, height=400, background_color="white").generate(spam_words).to_image().show()
plt.title("Spam WordCloud")

plt.subplot(1,2,2)
WordCloud(width=500, height=400, background_color="white").generate(ham_words).to_image().show()
plt.title("Ham WordCloud")

# 7) División train/test y vectorización con TF-IDF

In [ ]:
X = df["clean_msg"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print("Shape train TF-IDF:", X_train_tfidf.shape)

# 8) Modelo de Regresión Logística

In [ ]:
model = LogisticRegression(max_iter=1000, class_weight="balanced")
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)
y_prob = model.predict_proba(X_test_tfidf)[:,1]

# 9) Evaluación del modelo

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))

print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Ham","Spam"], yticklabels=["Ham","Spam"])
plt.title("Matriz de confusión")
plt.show()

# Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.plot(fpr, tpr, label="Logistic Regression (AUC = %.2f)" % roc_auc_score(y_test, y_prob))
plt.plot([0,1],[0,1],"k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

# 10) Predicción en nuevos datos (ejemplo)

In [ ]:
msgs = ["Win $5000 now by clicking this link!", 
        "Hi mom, I'll call you after work"]
msgs_clean = [clean_text(m) for m in msgs]
msgs_tfidf = tfidf.transform(msgs_clean)
preds = model.predict(msgs_tfidf)
print(list(zip(msgs, preds)))

# 11) Submission estilo Kaggle
En competencias de Kaggle se recibe un archivo `test.csv` y se debe crear `submission.csv`.  
Aquí lo simulamos: suponiendo que `test.csv` tiene columna `Id` y `message`.

In [ ]:
# Ejemplo simulado
fake_test = pd.DataFrame({
    "Id":[1,2],
    "message":["Claim your free prize!!!","Are we meeting tomorrow?"]
})
fake_test["clean_msg"] = fake_test["message"].apply(clean_text)
fake_test_tfidf = tfidf.transform(fake_test["clean_msg"])
fake_test["label"] = model.predict(fake_test_tfidf)

submission = fake_test[["Id","label"]]
submission.to_csv("submission.csv", index=False)
submission

# 12) Ideas para mejorar
- Optimizar hiperparámetros de la regresión logística (C, penalty).
- Probar representaciones más potentes (Word2Vec, embeddings, transformers).
- Usar modelos más avanzados (RandomForest, XGBoost, BERT).
- Balancear clases con técnicas como SMOTE si dataset está desbalanceado.
- Añadir limpieza más profunda (lematización, stemming).